# AutoInland Vehicle Insurance Claim Challenge
[link](https://zindi.africa/competitions/autoinland-vehicle-insurance-claim-challenge)

## Import libraries

In [1]:
from pathlib import Path

import altair as alt
import numpy as np
import pandas as pd

## Set some constants and define functions

In [2]:
MAIN_PATH = Path('.')
TRAIN = MAIN_PATH / 'data' / 'Train.csv'
TEST = MAIN_PATH / 'data' / 'Test.csv'
SUBMISSION = MAIN_PATH / 'data' / 'SampleSubmission.csv'
STATES = MAIN_PATH / 'data' / 'NigerianStateNames.csv'
VARS = MAIN_PATH / 'data' / 'VariableDefinitions.csv'

TARGET = 'ref_pm2_5'
FEATURES = []

TRAIN_COLOR = '#1f77b4'
TEST_COLOR = '#2ca02c'

In [3]:
def plot_number_of_unique_values(df, color):
    data = pd.DataFrame(df.nunique()).reset_index().rename(
        columns={'index': 'column_name', 0: 'unique_values'}
    )

    bars = alt.Chart(data).mark_bar().encode(
        x='unique_values:Q',
        y=alt.Y('column_name:O', sort='-x'),
        color=alt.ColorValue(color)
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3
    ).encode(
        text='unique_values:Q'
    )

    return bars + text

def compare_train_test(train, test, col, top_k=1000):
    data_l = train[col].value_counts(normalize=True)[:top_k].reset_index().rename(
        columns={'index': 'value', col: 'percentage'}
    )

    data_r = test[col].value_counts(normalize=True)[:top_k].reset_index().rename(
        columns={'index': 'value', col: 'percentage'}
    )
    data = data_l.merge(
        data_r,
        on='value',
        how='outer',
        suffixes=('_train', '_test')
    ).fillna(0)

    base = alt.Chart(data).properties(title=col)

    left = base.mark_bar().encode(
        x=alt.X('percentage_train:Q', sort=alt.SortOrder('descending')),
        y=alt.Y('value:O', axis=None),
        color=alt.ColorValue(TRAIN_COLOR)
    ).properties(title='Train')

    middle = base.encode(
        y=alt.Y('value:O', axis=None),
        text=alt.Text('value:O')
    ).mark_text().properties(width=100)

    right = base.mark_bar().encode(
        x='percentage_test:Q',
        y=alt.Y('value:O', axis=None),
        color=alt.ColorValue(TEST_COLOR)
    ).properties(title='Test')

    return alt.concat(left, middle, right)

## Read data

In [4]:
train = pd.read_csv(TRAIN, parse_dates=[
        'Policy Start Date',
        'Policy End Date',
        'First Transaction Date'
])
test = pd.read_csv(TEST, parse_dates=[
        'Policy Start Date',
        'Policy End Date',
        'First Transaction Date'
])
ss = pd.read_csv(SUBMISSION)

## Let's explore data

In [5]:
train.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      12079 non-null  object        
 1   Policy Start Date       12079 non-null  datetime64[ns]
 2   Policy End Date         12079 non-null  datetime64[ns]
 3   Gender                  11720 non-null  object        
 4   Age                     12079 non-null  int64         
 5   First Transaction Date  12079 non-null  datetime64[ns]
 6   No_Pol                  12079 non-null  int64         
 7   Car_Category            8341 non-null   object        
 8   Subject_Car_Colour      5117 non-null   object        
 9   Subject_Car_Make        9603 non-null   object        
 10  LGA_Name                5603 non-null   object        
 11  State                   5591 non-null   object        
 12  ProductName             12079 non-null  object

In [7]:
(train.isna().mean() * 100)

ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     2.972100
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              30.946270
Subject_Car_Colour        57.637222
Subject_Car_Make          20.498386
LGA_Name                  53.613710
State                     53.713056
ProductName                0.000000
target                     0.000000
dtype: float64

In [8]:
plot_number_of_unique_values(train.drop('ID', axis=1), TRAIN_COLOR)

alt.LayerChart(...)

In [9]:
plot_number_of_unique_values(test.drop('ID', axis=1), TEST_COLOR)

alt.LayerChart(...)

# Gender

In [10]:
compare_train_test(train, test, 'Gender')

alt.ConcatChart(...)

# Age

In [11]:
train['Age_outliers'] = ''
train.loc[(train['Age']>=14) & (train['Age']<=100), 'Age_outliers'] = 'Normal'
train.loc[(train['Age']<14), 'Age_outliers'] = 'Too low'
train.loc[(train['Age']>100), 'Age_outliers'] = 'Too high'

In [12]:
data = train.groupby('Age_outliers')['Age'].count().reset_index()
alt.Chart(data).mark_bar().encode(
    alt.X('Age', axis=alt.Axis(title='Count')),
    y='Age_outliers'
)

alt.Chart(...)

# Dates

In [13]:
for df in [train, test]:
        for n, col in enumerate([
            'Policy Start Date',
            'Policy End Date',
            'First Transaction Date'
        ]):
            df[f'{col} Year'] = df[col].dt.year
            df[f'{col} Month'] = df[col].dt.month
            df[f'{col} Day'] = df[col].dt.day
            df[f'{col} Weekday'] = df[col].dt.weekday

In [14]:
alt.vconcat(
    compare_train_test(train, test, 'Policy Start Date Month'),
    compare_train_test(train, test, 'Policy End Date Month'),
    compare_train_test(train, test, 'First Transaction Date Month')
)

alt.VConcatChart(...)

In [15]:
alt.vconcat(
    compare_train_test(train, test, 'Policy Start Date Day'),
    compare_train_test(train, test, 'Policy End Date Day'),
    compare_train_test(train, test, 'First Transaction Date Day')
)

alt.VConcatChart(...)

In [16]:
alt.vconcat(
    compare_train_test(train, test, 'Policy Start Date Weekday'),
    compare_train_test(train, test, 'Policy End Date Weekday'),
    compare_train_test(train, test, 'First Transaction Date Weekday')
)

alt.VConcatChart(...)

# No_Pol

In [17]:
compare_train_test(train, test, 'No_Pol')

alt.ConcatChart(...)

# Car Category

In [18]:
compare_train_test(train, test, 'Car_Category')

alt.ConcatChart(...)

# Color

In [19]:
compare_train_test(train, test, 'Subject_Car_Colour', top_k=10)

alt.ConcatChart(...)

# Car Maker

In [20]:
compare_train_test(train, test, 'Subject_Car_Make', top_k=15)

alt.ConcatChart(...)

# LGA

In [21]:
compare_train_test(train, test, 'LGA_Name', top_k=20)

alt.ConcatChart(...)

# States

In [22]:
compare_train_test(train, test, 'State', top_k=15)

alt.ConcatChart(...)

# Product Name

In [23]:
compare_train_test(train, test, 'ProductName')

alt.ConcatChart(...)